# Project 5

**Kai Lukowiak**

> _The goal of this project is give you practice beginning to work with a distributed recommender system. It is sufficient for this assignment to build out your application on a single node._

> Adapt one of your recommendation systems to work with Apache Spark and compare the performance with your previous iteration. Consider the efficiency of the system and the added complexity of using Spark. You may complete the assignment using PySpark (Python), SparkR (R) , sparklyr (R), or Scala.

In every project except the first I tried to implement Spark. However, I struggled with Java versions as well as spark locations. Eventually, I installed spark through sparklyr, the API has a command that install spark. I then deleted Java 10 from my computer and just had one version of 8. 

Now it works flawlesly. 

Most of my work was done in python, so I didn't use `recommenderlab` which is an R library. Instead, I used [Implicit](http://implicit.readthedocs.io/en/latest/). This comes with a built in ALS function. I'll compare these two.

---

## Implicit

First of all, a rehash of assignment 3 using Implicit. The notebook can be found [here](https://github.com/kaiserxc/DATA643/blob/master/Project3/Project3.ipynb)

Unfortunetly, Implicit doesn't run on my computer so I cann't compare the times that it takes to run. 

Also unfortunetly, I don't have the rmse values. However, I think the accuracy on this small scale will be similar.

While getting spark to work is more difficult at the start, the pyspark API is much easier to work with after the fact and makes working with recommenders easy and pleasurable. 

---

## Loading the Data:

In [16]:
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
ratings = pd.read_csv('../Project2/ml-latest-small/ratings.csv')
movies = pd.read_csv('../Project2/ml-latest-small/movies.csv')

In [18]:
## Import libraries
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark import SparkContext
from sklearn.metrics import mean_squared_error
import pandas as pd
from pyspark.sql import SQLContext

#### Initializing Spark

This step caused me so much pain over the entire course. But once it works, it's so simple.

In [19]:
## Initialize Spark
sc = SparkContext('local', 'project5.1')
sql_sc = SQLContext(sc)

In [20]:
## Load data
s_df = sql_sc.createDataFrame(ratings)

X_train, X_test = s_df.randomSplit([0.7, 0.3], seed=643)

#### Training

Really, this stop is so simple. It's not much more difficult than implementing linear regression.

In [21]:
## Training 
als = ALS(rank=10,
          maxIter=10,
          userCol='userId',
          itemCol='movieId',
          ratingCol='rating')
model = als.fit(X_train.select(['userId', 'movieId', 'rating']))

In [25]:
## Predictions

preds = model.transform(X_test.select(['userId', 'movieId']))

In [26]:
## Validate

# I don't like the spark evaluator
preds = preds.toPandas()
## To pandas

val = pd.merge(ratings, preds, on=['userId', 'movieId'])
val = val.dropna() # Worryingly, we have na values. Possibly something to do
# With the train test split??
rmse = mean_squared_error(val.rating, val.prediction)
print(rmse)
sc.stop()

0.8663015106653221


#### RMSE 
The RMSE is similar to what we have seen in previous assignments as well as on line for this data set. Tuning it could be an option, however, in some ways this would reduce the 'serendipity'.

In [31]:
combined = pd.merge(val, movies, on=['movieId'])
combined = combined[['userId', 'movieId', 'rating', 'prediction', 'title','genres']]
combined.sample(20)

,userId,movieId,rating,prediction,title,genres
24953,213,60072,2.5,2.668394,Wanted (2008),Action|Thriller
1188,659,736,3.0,2.993923,Twister (1996),Action|Adventure|Romance|Thriller
23946,165,30820,2.5,2.883806,"Woodsman, The (2004)",Drama
1731,468,1073,3.5,3.214813,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical
20342,442,48304,3.5,4.703349,Apocalypto (2006),Adventure|Drama|Thriller
20949,294,5693,2.0,2.937341,Saturday Night Fever (1977),Comedy|Drama|Romance
2698,247,2699,3.0,3.321315,Arachnophobia (1990),Comedy|Horror
18777,408,2006,4.0,2.651907,"Mask of Zorro, The (1998)",Action|Comedy|Romance
20350,553,49530,4.0,4.271254,Blood Diamond (2006),Action|Adventure|Crime|Drama|Thriller|War
12468,20,745,5.0,3.448871,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy


This sample honestly looks really good.

## Conclusion

> Please include in your conclusion: For your given recommender system’s data, algorithm(s), and (envisioned) implementation, at what point would you see moving to a distributed platform such as Spark becoming necessary?

As stated above, the Spark API is great for ALS, less good for something like SVD, although the capacity is still there, and horrible for setting up on a single cluster on my computer.

Honestly, for any ALS implementation in python, Spark is necessary off the bat. The Surprise package just doesn't cut it and nothing else is there.

Obviously, the distributed computing is another huge plus for spark. Any time the matrix size gets around the size of memory e.g., 16 Gigs you will need Spark. Even huge processors like on AWS might be able to fit it (128g) they are expensive and slower.

Another time spark might be used is in situations where recommendations are needed in real time, say after a movie is watched. Spark can help speed this up.

In the future, we plan on doing a final project on AWS where we will explore the benefits and costs more deeply.

## Thanks to:
### https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw
### https://medium.com/@connectwithghosh/simple-matrix-factorization-example-on-the-movielens-dataset-using-pyspark-9b7e3f567536
### https://dataplatform.cloud.ibm.com/exchange/public/entry/view/99b857815e69353c04d95daefb3b91fa